### 접근
- 어떤 홀수는 (2가 아닌)소수 a + 세미소수 b 로 나타낼 수 있다고 한다.
- 세미소수 리스트를 구하고, a+b = N_i 인 개수를 convolution으로 구하면 될 것 같다. 
- 세미 소수 리스트를 구하는 것이 참 곤란하다. $\Omicron(N^2)$ 으로 구하면 TLE가 날 것 같은데 다른방법이 없을까?
  - Golf의 풀이처럼 다항식을 제곱해서 푸는건 A+B에 해당하는 풀이다. 구하고자 하는것은 범위 내의 a*b가 해서 나올 수 있는 모든 수가 필요하다.
  - 세미소수 리스트를 모두 구하는 것이 아닐지도 모르겠다.
  - 문제에서 **짝수** 세미소수라는걸 이제 발견했다. 이러면 에라토스를 2에대해서 한번 더돌리면 세미소수 리스트를 구할 수 있다.

In [ ]:
import io, os, math
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

p = 469762049
def ntt(a, inv=False):
  n = len(a)
  j = 0
  for i in range(1,n):
    rev = n >> 1
    while j >= rev:
      j -= rev
      rev >>= 1
    j += rev
    if i < j:
      a[i], a[j] = a[j], a[i]

  step = 2
  while step <= n:
    half = step // 2
    u = pow(3,p//step,p)
    if inv: u = pow(u,p-2,p)
    for i in range(0, n, step):
      w = 1
      for j in range(i, i + half):
        v = a[j + half] * w
        a[j + half] = (a[j] - v)% p
        a[j] += v
        a[j] %= p
        w *= u
        w %= p
    step *= 2

  if inv:
    invn = p - (p-1) // n
    for i in range(n):
      a[i] = (a[i] * invn)%p
  return a
    
def ntt_conv(a, b):
  s = len(a) + len(b) - 1
  n = 1 << s.bit_length()
  a += [0 for _ in range(n - len(a))]
  b += [0 for _ in range(n - len(b))]
  ntt(a)
  ntt(b)
  for i in range(n):
    a[i] *= b[i]
  ntt(a,True)
  return a

def eratos(n):
  if n < 2: return set()
  n += 1
  P = [1] * (n//2)
  for i in range(3, math.isqrt(n) + 1, 2):
    if P[i//2]: P[i**2//2::i] = [0]*((n-i**2-1)//(2*i) + 1)

  return set([2] + [2*i+1 for i in range(1, n//2) if P[i]])  

MAX = 1_000_000
SIZE = 1 << math.ceil(math.log2(MAX))
def sol():
  sieve = eratos(MAX)
  P = [0] * SIZE
  Q = [0] * SIZE

  for v in sieve :
    P[v] = 1
    if 2*v > MAX : continue
    Q[2*v] = 1 #짝수 세미 소수
  P[2] = 0

  R = ntt_conv(P, Q)
  N = int(input())
  ans = []
  for _ in range(N):
    ans.append(R[int(input())])

  print('\n'.join(map(str,ans)))

sol()

### 자력솔
- 에라토스 범위를 잘못 설정해서 잔뜩 WA맞았다.